In [1]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import catboost
from catboost import CatBoostClassifier
import optuna 
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_parquet('train.parquet')
val = pd.read_parquet('val.parquet')
test = pd.read_parquet('test.parquet')

In [3]:
target = 'target'

# FIRST

In [4]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=3, random_seed=42)
model.fit(train.drop(columns=target), train[target], eval_set = (val.drop(columns=target), val[target]), verbose=100)

0:	learn: 0.6298857	test: 0.6441043	best: 0.6441043 (0)	total: 63.2ms	remaining: 1m 3s
100:	learn: 0.2230439	test: 0.3029602	best: 0.3029602 (100)	total: 127ms	remaining: 1.13s
200:	learn: 0.1339343	test: 0.2259582	best: 0.2259582 (200)	total: 164ms	remaining: 650ms
300:	learn: 0.0887578	test: 0.1869178	best: 0.1868728 (298)	total: 201ms	remaining: 466ms
400:	learn: 0.0623094	test: 0.1639398	best: 0.1638656 (396)	total: 233ms	remaining: 348ms
500:	learn: 0.0447381	test: 0.1474815	best: 0.1474815 (500)	total: 267ms	remaining: 266ms
600:	learn: 0.0343807	test: 0.1414494	best: 0.1403657 (590)	total: 299ms	remaining: 199ms
700:	learn: 0.0260732	test: 0.1371190	best: 0.1353437 (675)	total: 331ms	remaining: 141ms
800:	learn: 0.0208779	test: 0.1379913	best: 0.1353437 (675)	total: 362ms	remaining: 90ms
900:	learn: 0.0170486	test: 0.1405963	best: 0.1353437 (675)	total: 393ms	remaining: 43.2ms
999:	learn: 0.0149493	test: 0.1420813	best: 0.1353437 (675)	total: 424ms	remaining: 0us

bestTest = 0.1

In [5]:
print(f"TRAIN : {roc_auc_score(train[target], model.predict_proba(train.drop(columns=target))[:, 1])}")
print(f"VAL : {roc_auc_score(val[target], model.predict_proba(val.drop(columns=target))[:, 1])}")
print(f"TEST : {roc_auc_score(test[target], model.predict_proba(test.drop(columns=target))[:, 1])}")

TRAIN : 1.0
VAL : 0.9873610133708655
TEST : 0.9746920492721164


In [6]:
print(f"TRAIN : {f1_score(train[target], model.predict(train.drop(columns=target)))}")
print(f"VAL : {f1_score(val[target], model.predict(val.drop(columns=target)))}")
print(f"TEST : {f1_score(test[target], model.predict(test.drop(columns=target)))}")

TRAIN : 1.0
VAL : 0.9682151589242054
TEST : 0.9424083769633508


In [7]:
print(classification_report(test[target], model.predict(test.drop(columns=target))))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94       188
           1       0.94      0.95      0.94       190

    accuracy                           0.94       378
   macro avg       0.94      0.94      0.94       378
weighted avg       0.94      0.94      0.94       378



In [8]:
res = permutation_importance(model, train.drop(columns=target), train[target], n_repeats=10, random_state=42)

In [15]:
new_features = [list(train.drop(columns=target).columns)[i] for i in res['importances_mean'].argsort()[::1] if res['importances_mean'][i] - 2*res['importances_std'][i] > 0]



In [16]:
len(new_features)

11

In [17]:
new_features

['sex',
 'age',
 'chol',
 'thalachh',
 'slope',
 'trestbps',
 'restecg',
 'oldpeak',
 'ca',
 'cp',
 'thal']

In [18]:
train

,age,sex,cp,trestbps,chol,fbs,restecg,thalachh,exang,oldpeak,slope,ca,thal,target
1398,47,1,3,108,243,0,0,152,0,0.0,1,0,3,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
152,64,1,3,170,227,0,0,155,0,0.6,1,0,3,1
472,45,0,1,112,160,0,1,138,0,0.0,1,0,2,1
121,59,1,0,138,271,0,0,182,0,0.0,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,59,1,1,170,288,0,2,159,0,0.2,2,0,7,1
1306,52,1,0,108,233,1,1,147,0,0.1,2,3,3,1
1032,55,0,1,135,250,0,0,161,0,1.4,1,0,2,1
1839,70,0,2,160,269,0,1,112,1,2.9,1,1,3,0


# SECOND

In [19]:
train = train.drop(columns=['fbs', 'exang'])
val = val.drop(columns=['fbs', 'exang'])
test = test.drop(columns=['fbs', 'exang'])

In [20]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=3, random_seed=42)
model.fit(train.drop(columns=target), train[target], eval_set = (val.drop(columns=target), val[target]), verbose=100)

0:	learn: 0.6298857	test: 0.6441043	best: 0.6441043 (0)	total: 2.44ms	remaining: 2.44s
100:	learn: 0.2321796	test: 0.3044189	best: 0.3044189 (100)	total: 47ms	remaining: 418ms
200:	learn: 0.1481034	test: 0.2444828	best: 0.2438800 (197)	total: 79.9ms	remaining: 318ms
300:	learn: 0.0990293	test: 0.2001457	best: 0.2001457 (300)	total: 111ms	remaining: 258ms
400:	learn: 0.0672124	test: 0.1688962	best: 0.1687360 (399)	total: 142ms	remaining: 212ms
500:	learn: 0.0485114	test: 0.1561028	best: 0.1553731 (486)	total: 174ms	remaining: 173ms
600:	learn: 0.0356936	test: 0.1452511	best: 0.1452511 (600)	total: 206ms	remaining: 137ms
700:	learn: 0.0269617	test: 0.1422110	best: 0.1412684 (672)	total: 237ms	remaining: 101ms
800:	learn: 0.0213215	test: 0.1410357	best: 0.1404494 (790)	total: 269ms	remaining: 66.9ms
900:	learn: 0.0178062	test: 0.1429438	best: 0.1404494 (790)	total: 300ms	remaining: 33ms
999:	learn: 0.0153198	test: 0.1426403	best: 0.1404494 (790)	total: 331ms	remaining: 0us

bestTest = 0.1

In [21]:
print(f"TRAIN : {roc_auc_score(train[target], model.predict_proba(train.drop(columns=target))[:, 1])}")
print(f"VAL : {roc_auc_score(val[target], model.predict_proba(val.drop(columns=target))[:, 1])}")
print(f"TEST : {roc_auc_score(test[target], model.predict_proba(test.drop(columns=target))[:, 1])}")

TRAIN : 1.0
VAL : 0.9863476425052778
TEST : 0.9711926091825307


In [22]:
print(f"TRAIN : {f1_score(train[target], model.predict(train.drop(columns=target)))}")
print(f"VAL : {f1_score(val[target], model.predict(val.drop(columns=target)))}")
print(f"TEST : {f1_score(test[target], model.predict(test.drop(columns=target)))}")

TRAIN : 1.0
VAL : 0.9605911330049262
TEST : 0.9343832020997376


In [23]:
print(classification_report(test[target], model.predict(test.drop(columns=target))))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       188
           1       0.93      0.94      0.93       190

    accuracy                           0.93       378
   macro avg       0.93      0.93      0.93       378
weighted avg       0.93      0.93      0.93       378



In [28]:
importances = model.get_feature_importance(prettified=True)
a = importances.loc[importances['Importances'] >= 8, 'Feature Id'].tolist()

In [29]:
len(a)

8

In [30]:
importances

,Feature Id,Importances
0,thal,15.729118
1,cp,14.491009
2,oldpeak,8.957662
3,slope,8.860045
4,ca,8.810067
5,trestbps,8.766990
6,thalachh,8.492241
7,age,8.138874
8,chol,7.532217
9,restecg,6.339636


# THIRD

In [31]:
train = train.drop(columns=['sex', 'restecg', 'chol'])
val = val.drop(columns=['sex', 'restecg', 'chol'])
test = test.drop(columns=['sex', 'restecg', 'chol'])

In [32]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=3, random_seed=42)
model.fit(train.drop(columns=target), train[target], eval_set = (val.drop(columns=target), val[target]), verbose=100)

0:	learn: 0.6298857	test: 0.6441043	best: 0.6441043 (0)	total: 1.04ms	remaining: 1.04s
100:	learn: 0.2503086	test: 0.3214842	best: 0.3214842 (100)	total: 44.3ms	remaining: 394ms
200:	learn: 0.1759773	test: 0.2610128	best: 0.2610128 (200)	total: 77.7ms	remaining: 309ms
300:	learn: 0.1276305	test: 0.2179101	best: 0.2179101 (300)	total: 114ms	remaining: 264ms
400:	learn: 0.0920107	test: 0.1854333	best: 0.1854333 (400)	total: 148ms	remaining: 221ms
500:	learn: 0.0697787	test: 0.1689126	best: 0.1689126 (500)	total: 179ms	remaining: 178ms
600:	learn: 0.0550077	test: 0.1587213	best: 0.1587213 (600)	total: 210ms	remaining: 139ms
700:	learn: 0.0438893	test: 0.1537013	best: 0.1531763 (694)	total: 241ms	remaining: 103ms
800:	learn: 0.0356292	test: 0.1504991	best: 0.1504991 (800)	total: 280ms	remaining: 69.4ms
900:	learn: 0.0301278	test: 0.1491421	best: 0.1479260 (876)	total: 315ms	remaining: 34.6ms
999:	learn: 0.0264041	test: 0.1482264	best: 0.1467742 (966)	total: 349ms	remaining: 0us

bestTest =

In [33]:
print(f"TRAIN : {roc_auc_score(train[target], model.predict_proba(train.drop(columns=target))[:, 1])}")
print(f"VAL : {roc_auc_score(val[target], model.predict_proba(val.drop(columns=target))[:, 1])}")
print(f"TEST : {roc_auc_score(test[target], model.predict_proba(test.drop(columns=target))[:, 1])}")

TRAIN : 1.0
VAL : 0.985981703026038
TEST : 0.9726763717805151


In [34]:
print(f"TRAIN : {f1_score(train[target], model.predict(train.drop(columns=target)))}")
print(f"VAL : {f1_score(val[target], model.predict(val.drop(columns=target)))}")
print(f"TEST : {f1_score(test[target], model.predict(test.drop(columns=target)))}")

TRAIN : 1.0
VAL : 0.9519230769230769
TEST : 0.9347258485639687


In [35]:
print(classification_report(test[target], model.predict(test.drop(columns=target))))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       188
           1       0.93      0.94      0.93       190

    accuracy                           0.93       378
   macro avg       0.93      0.93      0.93       378
weighted avg       0.93      0.93      0.93       378



In [36]:
def objective(trial):
    param = {
        "n_estimators": trial.suggest_int('n_estimators', 300, 2000),
        "learning_rate": trial.suggest_float('learning_rate', 0.01, 0.3),
        "objective": trial.suggest_categorical('objective', ["Logloss", "CrossEntropy"]),
        "depth": trial.suggest_int('depth', 3, 10),
        "l2_leaf_reg": trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10),
        "colsample_bylevel": trial.suggest_float('colsample_bylevel', 0.01, 1.0),
        "min_child_samples": trial.suggest_float('min_child_samples', 100, 1000, step=100),
        "grow_policy": trial.suggest_categorical('grow_policy', ["SymmetricTree", "Depthwise", "Lossguide"]),
        "border_count": trial.suggest_categorical('border_count', [128, 256, 512, 1024]),
        "auto_class_weights": trial.suggest_categorical('auto_class_weight', [None, 'Balanced', 'SqrtBalanced'])
    }

    if param['objective'] == 'CrossEntropy':
        param['auto_class_weights'] = None

    model = CatBoostClassifier(**param, verbose = False, random_seed=42)

    model.fit(train.drop(columns=target), train[target], eval_set = (val.drop(columns=target), val[target]), early_stopping_rounds=50, plot=False)

    test_roc_auc = roc_auc_score(test[target], model.predict_proba(test.drop(columns=target))[:, 1])

    train_roc_auc = roc_auc_score(train[target], model.predict_proba(train.drop(columns=target))[:, 1])

    if train_roc_auc - test_roc_auc > 0.03:
        return 0
    else:
        return test_roc_auc

In [37]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2025-04-10 11:07:59,838] A new study created in memory with name: no-name-ed8cdf57-f751-4d37-80cb-7a936347448c
/var/folders/_k/vxqnb8117qb_1tpg9p98mtyc0000gn/T/ipykernel_13321/1751210426.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "l2_leaf_reg": trial.suggest_loguniform('l2_leaf_reg', 1e-2, 10),
[I 2025-04-10 11:08:00,341] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 1179, 'learning_rate': 0.018340138016913997, 'objective': 'Logloss', 'depth': 7, 'l2_leaf_reg': 0.011011234704944937, 'colsample_bylevel': 0.9442085437928284, 'min_child_samples': 400.0, 'grow_policy': 'Depthwise', 'border_count': 128, 'auto_class_weight': 'SqrtBalanced'}. Best is trial 0 with value: 0.0.
/var/folders/_k/vxqnb8117qb_1tpg9p98mtyc0000gn/T/ipykernel_13321/1751210426.py:7: FutureWarning: suggest_loguniform has been depre

In [38]:
print(study.best_params)

{'n_estimators': 369, 'learning_rate': 0.21401325569313503, 'objective': 'Logloss', 'depth': 10, 'l2_leaf_reg': 0.3040485290413335, 'colsample_bylevel': 0.7137746143457168, 'min_child_samples': 200.0, 'grow_policy': 'SymmetricTree', 'border_count': 128, 'auto_class_weight': None}


In [39]:
model = CatBoostClassifier(n_estimators = 369, learning_rate = 0.21401325569313503, objective = 'Logloss', depth = 10, l2_leaf_reg = 0.3040485290413335, colsample_bylevel = 0.7137746143457168, min_child_samples = 200.0, grow_policy = 'SymmetricTree', border_count = 128, auto_class_weights = None, random_seed=42)
model.fit(train.drop(columns=target), train[target], eval_set = (val.drop(columns=target), val[target]), verbose=100)





0:	learn: 0.4055532	test: 0.4357995	best: 0.4357995 (0)	total: 3.38ms	remaining: 1.24s
100:	learn: 0.0011517	test: 0.0722514	best: 0.0684035 (53)	total: 148ms	remaining: 392ms
200:	learn: 0.0011192	test: 0.0720388	best: 0.0684035 (53)	total: 275ms	remaining: 230ms
300:	learn: 0.0011172	test: 0.0720465	best: 0.0684035 (53)	total: 398ms	remaining: 89.8ms
368:	learn: 0.0011151	test: 0.0720340	best: 0.0684035 (53)	total: 479ms	remaining: 0us

bestTest = 0.06840349379
bestIteration = 53

Shrink model to first 54 iterations.


In [40]:
print(f"TRAIN : {roc_auc_score(train[target], model.predict_proba(train.drop(columns=target))[:, 1])}")
print(f"VAL : {roc_auc_score(val[target], model.predict_proba(val.drop(columns=target))[:, 1])}")
print(f"TEST : {roc_auc_score(test[target], model.predict_proba(test.drop(columns=target))[:, 1])}")

TRAIN : 1.0
VAL : 0.997466572836031
TEST : 0.9937569988801791


In [41]:
print(f"TRAIN : {f1_score(train[target], model.predict(train.drop(columns=target)))}")
print(f"VAL : {f1_score(val[target], model.predict(val.drop(columns=target)))}")
print(f"TEST : {f1_score(test[target], model.predict(test.drop(columns=target)))}")

TRAIN : 1.0
VAL : 0.9753694581280788
TEST : 0.9656992084432717


In [42]:
print(classification_report(test[target], model.predict(test.drop(columns=target))))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97       188
           1       0.97      0.96      0.97       190

    accuracy                           0.97       378
   macro avg       0.97      0.97      0.97       378
weighted avg       0.97      0.97      0.97       378



# with StandardScaler

In [4]:
x_train = train.drop(columns=target)
y_train = train[target]

x_val = val.drop(columns=target)
y_val = val[target]

x_test = test.drop(columns=target)
y_test = test[target]

In [5]:
scaler = StandardScaler()

In [6]:
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

In [7]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=3, random_seed=42)
model.fit(x_train_scaled, y_train, eval_set = (x_val_scaled, y_val), verbose=100)

0:	learn: 0.6298857	test: 0.6441043	best: 0.6441043 (0)	total: 59ms	remaining: 59s
100:	learn: 0.2230439	test: 0.3029602	best: 0.3029602 (100)	total: 126ms	remaining: 1.12s
200:	learn: 0.1339343	test: 0.2259582	best: 0.2259582 (200)	total: 163ms	remaining: 647ms
300:	learn: 0.0887578	test: 0.1869178	best: 0.1868728 (298)	total: 196ms	remaining: 456ms
400:	learn: 0.0623094	test: 0.1639398	best: 0.1638656 (396)	total: 232ms	remaining: 347ms
500:	learn: 0.0447381	test: 0.1474815	best: 0.1474815 (500)	total: 267ms	remaining: 266ms
600:	learn: 0.0343807	test: 0.1414494	best: 0.1403657 (590)	total: 304ms	remaining: 202ms
700:	learn: 0.0260732	test: 0.1371190	best: 0.1353437 (675)	total: 340ms	remaining: 145ms
800:	learn: 0.0208779	test: 0.1379913	best: 0.1353437 (675)	total: 371ms	remaining: 92.2ms
900:	learn: 0.0170486	test: 0.1405963	best: 0.1353437 (675)	total: 402ms	remaining: 44.2ms
999:	learn: 0.0149493	test: 0.1420813	best: 0.1353437 (675)	total: 434ms	remaining: 0us

bestTest = 0.135

In [8]:
print(f"TRAIN : {roc_auc_score(y_train, model.predict_proba(x_train_scaled)[:, 1])}")
print(f"VAL : {roc_auc_score(y_val, model.predict_proba(x_val_scaled)[:, 1])}")
print(f"TEST : {roc_auc_score(y_test, model.predict_proba(x_test_scaled)[:, 1])}")

TRAIN : 1.0
VAL : 0.9873610133708655
TEST : 0.9746920492721164


In [9]:
print(f"TRAIN : {f1_score(y_train, model.predict(x_train_scaled))}")
print(f"VAL : {f1_score(y_val, model.predict(x_val_scaled))}")
print(f"TEST : {f1_score(y_test, model.predict(x_test_scaled))}")

TRAIN : 1.0
VAL : 0.9682151589242054
TEST : 0.9424083769633508
